In [1]:
import pandas as pd
import numpy as np
import array
import json

import tensorflow as tf
import tensorflow.keras.preprocessing.text as kpt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation

Para este ejemplo se realizará la predicción utilizando solo el texto del dataset original.

In [2]:
train_df = pd.read_csv('../Data/train_processed.csv')
test_df = pd.read_csv('../Data/test_processed.csv')

In [3]:
# Creo los datos y los labels.
train_x = np.asarray(train_df['text_clean'])
test_x = np.asarray(test_df['text_clean'])
train_y = np.asarray(train_df['target'])

In [4]:
# Solo trabajo con las 3000 palabras mas populares del set de datos.
palabras_maximas = 3000

# Defino una función que me procese los datos.
def procesarDatos(arrayDeTextos):
    # Creo el tokenizador.
    tokenizador = Tokenizer(num_words = palabras_maximas)

    # Le paso los tweets al tokenizer.
    tokenizador.fit_on_texts(arrayDeTextos)

    # Lista de palabras y de indices que me da el tokenizer.
    diccionario = tokenizador.word_index    

    tweetsComoIndicesDePalabras = []

    # Función que convierte los tweets a un vector de indices.
    for tweet in arrayDeTextos:
        indicesDePalabrasDelTweet = [diccionario[palabra] for palabra in kpt.text_to_word_sequence(tweet)]
        tweetsComoIndicesDePalabras.append(indicesDePalabrasDelTweet)
    
    # Paso a array de numpy.
    tweetsComoIndicesDePalabras = np.asarray(tweetsComoIndicesDePalabras)

    # Creo una matriz del tipo one-hot encoding.
    matriz_x = tokenizer.sequences_to_matrix(tweetsComoIndicesDePalabras, mode='binary')

    return matriz_x

In [5]:
train_matriz_x = procesarDatos(train_x)
test_matriz_x = procesarDatos(test_x)

# La lista de labels es una lista de categorias: 1 tweet real; 0 tweet no real.
train_y = tf.keras.utils.to_categorical(train_y, 2)

NameError: name 'tokenizer' is not defined

In [ ]:
# Creo el modelo sequencial de tensorflow utilizando keras.

# HIPERPARÁMETROS A TENER EN CUENTA:
# 1 - Cantidad de palabras máximas.
# 2 - Número de salida del primer layer.
# 3 - Función de activación del primer layer.
# 4 - El porcentage de desactivación del primer dropout.
# 5 - Número de salida del segundo layer.
# 6 - Función de activación del segundo layer.
# 7 - El porcentage de desactivación del segundo dropout.
# 8 - Función de activación del layer final.
# 9 - La función loss del modelo.
# 10 - La función optimizer del modelo.


# Red neuronal que es un stack de layer ejecutados en orden.
modelo = Sequential() 

# El primer y último layer son de los más importantes ya que son la entrada y la salida.
# El primer layer tiene entrada de 3000 que es la cantidad máxima de palabras.
# 512 es la salida de este layer y la función de activación/maximización es relu.
modelo.add(Dense(512, input_shape=(palabras_maximas,), activation='relu'))

# Desactivo neuronas con el fin de no caer en overfitting.
modelo.add(Dropout(0.2))

# Capa intermedia que tiene como salida 256 neuronas y la función de activación es sigmoid.
modelo.add(Dense(256, activation='sigmoid'))

# Desactivo neuronas con el fin de no caer en overfitting.
modelo.add(Dropout(0.2))

# Layer de salida, tiene solo 2 que la cantidad de posibilidades que tiene el target.
# Su función de activación es softmax.
modelo.add(Dense(2, activation='softmax'))

# Finalmente compilo el modelo.
# Cómo métrica le pido que me devuelva el accuracy, la precisión y el recall 
# para poder calcular con estas últimas dos la puntuación F1.
modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])


In [ ]:
# Entreno el modelo.

# HIPERPARÁMETROS A TENER EN CUENTA:
# 1 - Batch_size.
# 2 - Epochs.
# 3 - Split.

# Entro el modelo tomando los datos de a 32 registros.
# Hago 20 iteraciones.
# Valido con el 20% de los datos.
modelo.fit(matriz_x, train_y, batch_size = 32, epochs = 20, validation_split = 0.2)

In [ ]:
predicciones = modelo.predict(test_matriz_x)
categorias = array.array('i')

for prediccion in predicciones:
    categorias.append(np.argmax(prediccion))

In [ ]:
# Genero un dataframe de pandas y lo guardo.
test_prediccion = test_df['id'].to_frame()
test_prediccion['target'] = pd.Series(categorias)
test_prediccion.to_csv('tensorFlow.csv', index = False)